In [1]:
# -*- coding: utf-8 -*-
import warnings
warnings.filterwarnings('ignore')

import csv, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.externals import joblib
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

pd.set_option('display.max_columns', 500)
curr_dir = '/home/souche/qiongjiu/hgc/'

In [2]:
data_train = pd.read_csv(curr_dir+"2020-11-02版零售数据data_train2.csv", header=0)
data_test = pd.read_csv(curr_dir+"2020-11-02版零售数据data_test2.csv", header=0)
print("按级别划分前训练数据数目为:%d" % data_train.shape[0])
print("按级别划分前训练数据包含的车型数为:%d" % len(data_train['model_code'].unique()))
print("按级别划分前测试数据数目为:%d" % data_test.shape[0])
print("按级别划分前测试数据包含的车型数为:%d" % len(data_test['model_code'].unique()))

按级别划分前训练数据数目为:1359928
按级别划分前训练数据包含的车型数为:29411
按级别划分前测试数据数目为:341915
按级别划分前测试数据包含的车型数为:25333


In [3]:
car = pd.read_csv(curr_dir+"2020-11-02版车型参数及独热编码.csv", header = 0, low_memory=False)
car=car.drop(['model_year','new_car_price', 'driving_mode','gear_box_type','country_id',
 'import_type','intake_type', 'fuel_form','quality_mile', 'quality_year',
 'year_1','year_2','year_3','year_4','year_5','year_6','year_7','year_8',
 'year_9', 'year_10','year_11','year_12','year_13','year_14','year_15',
 'year_16', 'car_body','rate','rate_count'],axis=1)

data_train = pd.merge(data_train, car, how="inner", on="model_code")
data_test = pd.merge(data_test, car, how="inner", on="model_code")
print(data_train.shape)
print(data_test.shape)

(1359928, 211)
(341915, 211)


In [5]:
vec = pd.read_csv(curr_dir+"2020-11-02版-TF-IDF权重的词向量.csv", dtype={'model_code': str}, header = 0)
vec = vec.drop(['model_name'], axis=1)
print(vec.shape)

(65250, 101)


In [7]:
data_train = pd.merge(data_train, vec, how="inner", on="model_code")
data_test = pd.merge(data_test, vec, how="inner", on="model_code")

In [8]:
print(data_train.shape)
print(data_test.shape)

(1359928, 311)
(341915, 311)


### 先计算B级车的误差中位数

In [9]:
train_B = data_train[(data_train['level']=="B") | 
                     (data_train['level']=="C") | 
                     (data_train['level']=="D") | 
                     (data_train['level']=="E")]
print("筛选出B级以上的车训练集数量为:%d" % train_B.shape[0])
print("筛选出B级以上的车训练集--车型数量为:%d" % len(train_B['model_code'].unique()))
print("<<-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*>>")

test_B = data_test[(data_test['level']=="B") | 
                   (data_test['level']=="C") | 
                   (data_test['level']=="D") | 
                   (data_test['level']=="E")]
print("筛选出B级以上的车测试集数量为:%d" % test_B.shape[0])
print("筛选出B级以上的车测试集--车型数量为:%d" % len(test_B['model_code'].unique()))

筛选出B级以上的车训练集数量为:663741
筛选出B级以上的车训练集--车型数量为:10985
<<-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*>>
筛选出B级以上的车测试集数量为:166629
筛选出B级以上的车测试集--车型数量为:9785


In [11]:
# 从train_A中再抽取10%的数据作为validation set
val_B = train_B.sample(frac=0.1, random_state=0)
train_B = train_B.drop(val_B.index)
print(val_B.shape, train_B.shape)

print(train_B['level'].unique())
print(val_B['level'].unique())
print(test_B['level'].unique())

(59737, 311) (537630, 311)
['B' 'C' 'D' 'E']
['C' 'B' 'D' 'E']
['B' 'C' 'D' 'E']


In [12]:
# 将训练集测试集打乱
train_B = shuffle(train_B)
val_B = shuffle(val_B)
test_B = shuffle(test_B)

train_X_B = train_B.drop(['car_id', 'brand_code', 'brand_name', 'series_code', 
                          'series_name', 'model_code', 'model_name',  'color_name', 
                          'first_license_plate_date', 'province_name', 'city_name', 
                          'price', 'pay_time', 'residual', 'mileage_std', 'license_month', 
                          'new_car_price', 'count', 'series_level', 'level','level0','level1',
                          'level2','level3','level4','level5','level6','level7','level8'], axis=1)
train_y_B = train_B['residual']

val_X_B = val_B.drop(['car_id', 'brand_code', 'brand_name', 'series_code', 
                          'series_name', 'model_code', 'model_name',  'color_name', 
                          'first_license_plate_date', 'province_name', 'city_name', 
                          'price', 'pay_time', 'residual', 'mileage_std', 'license_month', 
                          'new_car_price', 'count', 'series_level', 'level','level0','level1',
                          'level2','level3','level4','level5','level6','level7','level8'], axis=1)
val_y_B = val_B['residual']

test_X_B = test_B.drop(['car_id', 'brand_code', 'brand_name', 'series_code', 
                          'series_name', 'model_code', 'model_name',  'color_name', 
                          'first_license_plate_date', 'province_name', 'city_name', 
                          'price', 'pay_time', 'residual', 'mileage_std', 'license_month', 
                          'new_car_price', 'count', 'series_level', 'level','level0','level1',
                          'level2','level3','level4','level5','level6','level7','level8'], axis=1)
test_y_B = test_B['residual']

print(train_X_B.shape, train_y_B.shape)
print(val_X_B.shape, val_y_B.shape)
print(test_X_B.shape, test_y_B.shape)

(537630, 282) (537630,)
(59737, 282) (59737,)
(166629, 282) (166629,)


In [17]:
fearture_columns = ['days', 'license_year', 'model_year', 'rate', 'rate_count', 'per_mile', 'year_err', 'province_name0', 'province_name1', 'province_name2', 'province_name3', 'province_name4', 'province_name5', 'province_name6', 'province_name7', 'province_name8', 'province_name9', 'province_name10', 'province_name11', 'province_name12', 'province_name13', 
 'province_name14', 'province_name15', 'province_name16', 'province_name17', 'province_name18', 'province_name19', 'province_name20', 'province_name21', 'province_name22', 'province_name23', 'province_name24', 'province_name25', 'province_name26', 'province_name27', 'province_name28', 'province_name29', 'province_name30', 'keep_value', 'quality', 'mile_log',
 'license_month_log', 'new_car_price_log', 'wheel_base', 'length', 'height', 'width', 'max_torque', 'max_power', 'engine_volume_l', 'cylinder_number', 'seat_number_top', 'driving_mode0', 'driving_mode1', 'driving_mode2', 'driving_mode3', 'driving_mode4', 'driving_mode5', 'driving_mode6', 'driving_mode7', 'driving_mode8', 'driving_mode9', 'gear_box_type0', 'gear_box_type1', 'gear_box_type2', 'gear_box_type3', 'gear_box_type4',
 'gear_box_type5', 'gear_box_type6', 'gear_box_type7', 'gear_box_type8', 'gear_box_type9', 'gear_box_type10', 'country_id0', 'country_id1', 'country_id2', 'country_id3', 'country_id4', 'country_id5', 'country_id6', 'country_id7', 'country_id8', 'country_id9', 'country_id10', 'country_id11', 'country_id12', 'country_id13', 'country_id14', 'import_type0', 'import_type1', 'intake_type0', 'intake_type1', 'intake_type2', 'intake_type3',
 'intake_type4', 'intake_type5', 'intake_type6', 'intake_type7', 'fuel_form0', 'fuel_form1', 'fuel_form2', 'fuel_form3', 'fuel_form4', 'fuel_form5', 'fuel_form6', 'fuel_form7', 'fuel_form8', 'car_body0', 'car_body1', 'car_body2', 'car_body3', 'car_body4', 'car_body5', 'car_body6', 'car_body7', 'car_body8', 'car_body9', 'car_body10', 'car_body11', 'series_level0', 'series_level1', 'series_level2', 'series_level3', 'series_level4', 
 'series_level5', 'series_level6', 'series_level7', 'series_level8', 'series_level9', 'series_level10', 'series_level11', 'series_level12', 'series_level13', 'series_level14', 'series_level15', 'series_level16', 'series_level17', 'series_level18', 'series_level19', 'series_level20', 'series_level21', 'series_level22', 'series_level23', 'series_level24', 'series_level25', 'series_level26', 'series_level27', 'series_level28', 'series_level29',
 'series_level30', 'series_level31', 'series_level32', 'series_level33', 'series_level34', 'series_level35', 'series_level36', 'series_level37', 'series_level38', 'series_level39', 'series_level40', 'series_level41', 'series_level42', 'series_level43', 'series_level44', 'series_level45', 'series_level46', 'series_level47', 'series_level48', 'series_level49', 'series_level50', 'series_level51', 'series_level52', 'series_level53', 'series_level54', 
 'series_level55', 'series_level56', 'series_level57', 'series_level58', 'series_level59', 'series_level60', 'series_level61', 'series_level62', 'wordvec0', 'wordvec1', 'wordvec2', 'wordvec3', 'wordvec4', 'wordvec5', 'wordvec6', 'wordvec7', 'wordvec8', 'wordvec9', 'wordvec10', 'wordvec11', 'wordvec12', 'wordvec13', 'wordvec14', 'wordvec15', 'wordvec16', 'wordvec17', 'wordvec18', 'wordvec19', 'wordvec20', 'wordvec21', 'wordvec22', 'wordvec23', 'wordvec24', 
 'wordvec25', 'wordvec26', 'wordvec27', 'wordvec28', 'wordvec29', 'wordvec30', 'wordvec31', 'wordvec32', 'wordvec33', 'wordvec34', 'wordvec35', 'wordvec36', 'wordvec37', 'wordvec38', 'wordvec39', 'wordvec40', 'wordvec41', 'wordvec42', 'wordvec43', 'wordvec44', 'wordvec45', 'wordvec46', 'wordvec47', 'wordvec48', 'wordvec49', 'wordvec50', 'wordvec51', 'wordvec52', 'wordvec53', 'wordvec54', 'wordvec55', 'wordvec56', 'wordvec57', 'wordvec58', 'wordvec59', 'wordvec60', 
 'wordvec61', 'wordvec62', 'wordvec63', 'wordvec64', 'wordvec65', 'wordvec66', 'wordvec67', 'wordvec68', 'wordvec69', 'wordvec70', 'wordvec71', 'wordvec72', 'wordvec73', 'wordvec74', 'wordvec75', 'wordvec76', 'wordvec77', 'wordvec78', 'wordvec79', 'wordvec80', 'wordvec81', 'wordvec82', 'wordvec83', 'wordvec84', 'wordvec85', 'wordvec86', 'wordvec87', 'wordvec88', 'wordvec89', 'wordvec90', 'wordvec91', 'wordvec92', 'wordvec93', 'wordvec94', 'wordvec95', 'wordvec96', 
 'wordvec97', 'wordvec98', 'wordvec99']

label = 'residual'

In [19]:
import lightgbm as lgb
# 零售 B 级车的参数：
gbm = lgb.LGBMRegressor(boosting_type = 'gbdt', 
                        objective = 'regression', 
                        learning_rate = 0.05, 
                        n_estimators = 3000, 
                        max_depth = 8, 
                        num_leaves = 150, 
                        subsample = 1.0, 
                        colsample_bytree = 0.3, 
                        min_child_samples = 80, 
                        n_jobs = 8, 
                        verbose = 1)

gbm.fit(train_B[fearture_columns], 
        train_B[label],
        eval_set = (val_B[fearture_columns], val_B[label]),
        eval_metric = {'l1','l2'}, 
        early_stopping_rounds = 50, 
        verbose=True)

[1]	valid_0's l2: 0.038805	valid_0's l1: 0.163013
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's l2: 0.0369595	valid_0's l1: 0.159141
[3]	valid_0's l2: 0.0338281	valid_0's l1: 0.152154
[4]	valid_0's l2: 0.0310317	valid_0's l1: 0.145661
[5]	valid_0's l2: 0.0285058	valid_0's l1: 0.13952
[6]	valid_0's l2: 0.0263543	valid_0's l1: 0.134014
[7]	valid_0's l2: 0.0242548	valid_0's l1: 0.128456
[8]	valid_0's l2: 0.0223616	valid_0's l1: 0.123233
[9]	valid_0's l2: 0.0206535	valid_0's l1: 0.118337
[10]	valid_0's l2: 0.0196196	valid_0's l1: 0.115269
[11]	valid_0's l2: 0.0186545	valid_0's l1: 0.112365
[12]	valid_0's l2: 0.0177815	valid_0's l1: 0.109647
[13]	valid_0's l2: 0.0169601	valid_0's l1: 0.10702
[14]	valid_0's l2: 0.0157342	valid_0's l1: 0.102994
[15]	valid_0's l2: 0.0146362	valid_0's l1: 0.0992139
[16]	valid_0's l2: 0.0136154	valid_0's l1: 0.095597
[17]	valid_0's l2: 0.013044	valid_0's l1: 0.0935153
[18]	valid_0's l2: 0.0122228	valid_0's l1: 0.0904104
[19]	valid_0

[157]	valid_0's l2: 0.00233123	valid_0's l1: 0.0356655
[158]	valid_0's l2: 0.00232708	valid_0's l1: 0.0356263
[159]	valid_0's l2: 0.00232627	valid_0's l1: 0.0356184
[160]	valid_0's l2: 0.00232494	valid_0's l1: 0.0356056
[161]	valid_0's l2: 0.00232302	valid_0's l1: 0.0355876
[162]	valid_0's l2: 0.00231552	valid_0's l1: 0.0355215
[163]	valid_0's l2: 0.00231012	valid_0's l1: 0.035473
[164]	valid_0's l2: 0.00230866	valid_0's l1: 0.0354591
[165]	valid_0's l2: 0.00230683	valid_0's l1: 0.0354409
[166]	valid_0's l2: 0.00230583	valid_0's l1: 0.0354322
[167]	valid_0's l2: 0.0023013	valid_0's l1: 0.0353913
[168]	valid_0's l2: 0.00229712	valid_0's l1: 0.0353524
[169]	valid_0's l2: 0.00229598	valid_0's l1: 0.0353413
[170]	valid_0's l2: 0.00229111	valid_0's l1: 0.0352984
[171]	valid_0's l2: 0.0022878	valid_0's l1: 0.0352674
[172]	valid_0's l2: 0.00228453	valid_0's l1: 0.0352402
[173]	valid_0's l2: 0.00228344	valid_0's l1: 0.0352298
[174]	valid_0's l2: 0.00228228	valid_0's l1: 0.035218
[175]	valid_0'

[310]	valid_0's l2: 0.00208215	valid_0's l1: 0.0332877
[311]	valid_0's l2: 0.00208136	valid_0's l1: 0.0332789
[312]	valid_0's l2: 0.00207954	valid_0's l1: 0.0332629
[313]	valid_0's l2: 0.00207779	valid_0's l1: 0.0332468
[314]	valid_0's l2: 0.00207721	valid_0's l1: 0.0332412
[315]	valid_0's l2: 0.00207677	valid_0's l1: 0.0332365
[316]	valid_0's l2: 0.00207576	valid_0's l1: 0.0332273
[317]	valid_0's l2: 0.00207491	valid_0's l1: 0.0332196
[318]	valid_0's l2: 0.00207451	valid_0's l1: 0.0332149
[319]	valid_0's l2: 0.00207408	valid_0's l1: 0.0332104
[320]	valid_0's l2: 0.00207366	valid_0's l1: 0.0332075
[321]	valid_0's l2: 0.00207285	valid_0's l1: 0.0331981
[322]	valid_0's l2: 0.00207188	valid_0's l1: 0.0331889
[323]	valid_0's l2: 0.00207102	valid_0's l1: 0.0331808
[324]	valid_0's l2: 0.00207028	valid_0's l1: 0.0331735
[325]	valid_0's l2: 0.00206901	valid_0's l1: 0.0331608
[326]	valid_0's l2: 0.00206841	valid_0's l1: 0.0331543
[327]	valid_0's l2: 0.00206794	valid_0's l1: 0.0331491
[328]	vali

[461]	valid_0's l2: 0.00197718	valid_0's l1: 0.0322422
[462]	valid_0's l2: 0.00197673	valid_0's l1: 0.0322379
[463]	valid_0's l2: 0.00197622	valid_0's l1: 0.0322325
[464]	valid_0's l2: 0.00197526	valid_0's l1: 0.0322227
[465]	valid_0's l2: 0.0019745	valid_0's l1: 0.0322146
[466]	valid_0's l2: 0.00197441	valid_0's l1: 0.0322137
[467]	valid_0's l2: 0.00197351	valid_0's l1: 0.0322036
[468]	valid_0's l2: 0.00197299	valid_0's l1: 0.0321978
[469]	valid_0's l2: 0.00197265	valid_0's l1: 0.0321943
[470]	valid_0's l2: 0.00197228	valid_0's l1: 0.0321909
[471]	valid_0's l2: 0.00197178	valid_0's l1: 0.0321856
[472]	valid_0's l2: 0.00197134	valid_0's l1: 0.0321816
[473]	valid_0's l2: 0.00197036	valid_0's l1: 0.0321717
[474]	valid_0's l2: 0.00196985	valid_0's l1: 0.0321662
[475]	valid_0's l2: 0.00196946	valid_0's l1: 0.0321625
[476]	valid_0's l2: 0.00196889	valid_0's l1: 0.0321564
[477]	valid_0's l2: 0.00196841	valid_0's l1: 0.0321502
[478]	valid_0's l2: 0.00196765	valid_0's l1: 0.0321432
[479]	valid

[615]	valid_0's l2: 0.00191733	valid_0's l1: 0.0316229
[616]	valid_0's l2: 0.00191689	valid_0's l1: 0.0316175
[617]	valid_0's l2: 0.00191663	valid_0's l1: 0.0316149
[618]	valid_0's l2: 0.00191633	valid_0's l1: 0.031612
[619]	valid_0's l2: 0.00191595	valid_0's l1: 0.0316077
[620]	valid_0's l2: 0.00191561	valid_0's l1: 0.0316039
[621]	valid_0's l2: 0.00191503	valid_0's l1: 0.0315988
[622]	valid_0's l2: 0.00191477	valid_0's l1: 0.0315964
[623]	valid_0's l2: 0.00191441	valid_0's l1: 0.0315922
[624]	valid_0's l2: 0.00191426	valid_0's l1: 0.0315905
[625]	valid_0's l2: 0.00191378	valid_0's l1: 0.031585
[626]	valid_0's l2: 0.00191355	valid_0's l1: 0.0315819
[627]	valid_0's l2: 0.00191326	valid_0's l1: 0.0315779
[628]	valid_0's l2: 0.00191297	valid_0's l1: 0.0315742
[629]	valid_0's l2: 0.00191261	valid_0's l1: 0.0315703
[630]	valid_0's l2: 0.00191172	valid_0's l1: 0.0315594
[631]	valid_0's l2: 0.00191148	valid_0's l1: 0.0315573
[632]	valid_0's l2: 0.00191138	valid_0's l1: 0.0315558
[633]	valid_

...

[2411]	valid_0's l2: 0.00177961	valid_0's l1: 0.0301266
[2412]	valid_0's l2: 0.0017796	valid_0's l1: 0.0301266
[2413]	valid_0's l2: 0.00177963	valid_0's l1: 0.0301269
[2414]	valid_0's l2: 0.0017796	valid_0's l1: 0.0301263
[2415]	valid_0's l2: 0.00177958	valid_0's l1: 0.0301261
[2416]	valid_0's l2: 0.00177961	valid_0's l1: 0.0301262
[2417]	valid_0's l2: 0.00177964	valid_0's l1: 0.0301262
[2418]	valid_0's l2: 0.00177962	valid_0's l1: 0.0301259
[2419]	valid_0's l2: 0.00177964	valid_0's l1: 0.0301265
[2420]	valid_0's l2: 0.00177958	valid_0's l1: 0.0301261
[2421]	valid_0's l2: 0.00177957	valid_0's l1: 0.030126
[2422]	valid_0's l2: 0.00177952	valid_0's l1: 0.0301256
[2423]	valid_0's l2: 0.00177952	valid_0's l1: 0.0301255
[2424]	valid_0's l2: 0.00177941	valid_0's l1: 0.0301242
[2425]	valid_0's l2: 0.00177937	valid_0's l1: 0.0301236
[2426]	valid_0's l2: 0.00177936	valid_0's l1: 0.0301235
[2427]	valid_0's l2: 0.00177937	valid_0's l1: 0.0301234
[2428]	valid_0's l2: 0.00177932	valid_0's l1: 0.030

[2561]	valid_0's l2: 0.0017779	valid_0's l1: 0.0301067
[2562]	valid_0's l2: 0.00177788	valid_0's l1: 0.0301067
[2563]	valid_0's l2: 0.00177793	valid_0's l1: 0.0301071
[2564]	valid_0's l2: 0.00177793	valid_0's l1: 0.0301073
[2565]	valid_0's l2: 0.00177792	valid_0's l1: 0.0301073
[2566]	valid_0's l2: 0.00177791	valid_0's l1: 0.030107
[2567]	valid_0's l2: 0.00177787	valid_0's l1: 0.0301064
[2568]	valid_0's l2: 0.00177787	valid_0's l1: 0.0301063
[2569]	valid_0's l2: 0.00177786	valid_0's l1: 0.0301063
[2570]	valid_0's l2: 0.00177788	valid_0's l1: 0.0301067
[2571]	valid_0's l2: 0.00177785	valid_0's l1: 0.0301063
[2572]	valid_0's l2: 0.00177786	valid_0's l1: 0.0301064
[2573]	valid_0's l2: 0.00177787	valid_0's l1: 0.0301064
[2574]	valid_0's l2: 0.00177786	valid_0's l1: 0.0301066
[2575]	valid_0's l2: 0.0017779	valid_0's l1: 0.0301068
[2576]	valid_0's l2: 0.00177792	valid_0's l1: 0.030107
[2577]	valid_0's l2: 0.0017779	valid_0's l1: 0.0301067
[2578]	valid_0's l2: 0.0017779	valid_0's l1: 0.030106

[2711]	valid_0's l2: 0.00177624	valid_0's l1: 0.0300838
[2712]	valid_0's l2: 0.00177623	valid_0's l1: 0.0300835
[2713]	valid_0's l2: 0.0017762	valid_0's l1: 0.0300834
[2714]	valid_0's l2: 0.0017762	valid_0's l1: 0.0300831
[2715]	valid_0's l2: 0.00177617	valid_0's l1: 0.0300829
[2716]	valid_0's l2: 0.00177613	valid_0's l1: 0.0300826
[2717]	valid_0's l2: 0.00177611	valid_0's l1: 0.0300823
[2718]	valid_0's l2: 0.00177611	valid_0's l1: 0.0300823
[2719]	valid_0's l2: 0.00177611	valid_0's l1: 0.0300824
[2720]	valid_0's l2: 0.00177612	valid_0's l1: 0.0300824
[2721]	valid_0's l2: 0.00177614	valid_0's l1: 0.0300826
[2722]	valid_0's l2: 0.00177617	valid_0's l1: 0.0300827
[2723]	valid_0's l2: 0.00177618	valid_0's l1: 0.0300826
[2724]	valid_0's l2: 0.00177621	valid_0's l1: 0.0300827
[2725]	valid_0's l2: 0.00177623	valid_0's l1: 0.0300827
[2726]	valid_0's l2: 0.00177621	valid_0's l1: 0.0300824
[2727]	valid_0's l2: 0.00177621	valid_0's l1: 0.0300821
[2728]	valid_0's l2: 0.00177621	valid_0's l1: 0.03

[2858]	valid_0's l2: 0.00177524	valid_0's l1: 0.0300699
[2859]	valid_0's l2: 0.00177526	valid_0's l1: 0.0300701
[2860]	valid_0's l2: 0.00177527	valid_0's l1: 0.0300701
[2861]	valid_0's l2: 0.00177527	valid_0's l1: 0.03007
[2862]	valid_0's l2: 0.00177527	valid_0's l1: 0.0300699
[2863]	valid_0's l2: 0.00177525	valid_0's l1: 0.0300697
[2864]	valid_0's l2: 0.00177525	valid_0's l1: 0.0300695
[2865]	valid_0's l2: 0.00177524	valid_0's l1: 0.0300692
[2866]	valid_0's l2: 0.00177525	valid_0's l1: 0.0300693
[2867]	valid_0's l2: 0.00177526	valid_0's l1: 0.0300695
[2868]	valid_0's l2: 0.00177524	valid_0's l1: 0.0300692
[2869]	valid_0's l2: 0.00177526	valid_0's l1: 0.0300692
[2870]	valid_0's l2: 0.00177527	valid_0's l1: 0.0300694
[2871]	valid_0's l2: 0.00177529	valid_0's l1: 0.0300695
[2872]	valid_0's l2: 0.00177523	valid_0's l1: 0.0300688
[2873]	valid_0's l2: 0.00177519	valid_0's l1: 0.0300687
[2874]	valid_0's l2: 0.0017752	valid_0's l1: 0.0300687
[2875]	valid_0's l2: 0.00177512	valid_0's l1: 0.030

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.3,
       learning_rate=0.05, max_depth=8, min_child_samples=80,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=3000,
       n_jobs=8, num_leaves=150, objective='regression', random_state=None,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=0, verbose=1)

In [35]:
# 测试集误差打印
# 以误差 查看精度
def print_test_result(ypred, test_y):
    err3, err5, err8, err10 = 0, 0, 0, 0
    test_y = np.array(test_y)
    ypred = np.array(ypred)
    size = len(ypred)
    for i in range(size):
        err = abs(ypred[i] - test_y[i])
        if(err<=0.03):
            err3 +=1
        if(err<=0.05):
            err5 += 1
        if(err<=0.08):
            err8 +=1
        if(err<=0.1):
            err10 += 1
    print('预测误差在百分之 3以内数目为：%5d,所占比例为：%.4f%%'% (err3, err3/size*100))
    print('预测误差在百分之 5以内数目为：%5d,所占比例为：%.4f%%'% (err5, err5/size*100))
    print('预测误差在百分之 8以内数目为：%5d,所占比例为：%.4f%%'% (err8, err8/size*100))
    print('预测误差在百分之10以内数目为：%5d,所占比例为：%.4f%%'% (err10, err10/size*100))

In [21]:
ypred = gbm.predict(test_B[fearture_columns])
print_test_result(ypred, test_B[label])

预测误差在百分之 3以内数目为：104371,所占比例为：62.6368%
预测误差在百分之 5以内数目为：138418,所占比例为：83.0696%
预测误差在百分之 8以内数目为：157347,所占比例为：94.4295%
预测误差在百分之10以内数目为：161726,所占比例为：97.0575%


In [27]:
test_B['pred'] = gbm.predict(test_B[fearture_columns])


In [36]:
print_test_result(test_B.loc[test_B['pay_time'] > '2020-09-01']['pred'],test_B.loc[test_B['pay_time'] > '2020-09-01'][label])


预测误差在百分之 3以内数目为： 5737,所占比例为：67.5497%
预测误差在百分之 5以内数目为： 7372,所占比例为：86.8009%
预测误差在百分之 8以内数目为： 8148,所占比例为：95.9378%
预测误差在百分之10以内数目为： 8305,所占比例为：97.7864%


In [37]:
by_model = test_B[['model_code', 'count', 'residual']]
by_model['pre_residual'] = ypred
by_model['pre_err'] = abs(by_model['residual'] - by_model['pre_residual'])
by_model['percent_err'] = by_model['pre_err']/by_model['residual']

data_result = pd.DataFrame(columns={'model_code':str, 'median_res':float})

data_result['model_code'] = list(by_model.groupby(['model_code'])['percent_err'].quantile(0.5).index)
data_result['median_res'] = list(by_model.groupby(['model_code'])['percent_err'].quantile(0.5))

# 以误差率 看精度  
print(data_result[data_result['median_res']<=0.03].shape[0], 
      data_result[data_result['median_res']<=0.03].shape[0]/data_result.shape[0])
print(data_result[data_result['median_res']<=0.05].shape[0], 
      data_result[data_result['median_res']<=0.05].shape[0]/data_result.shape[0])
print(data_result[data_result['median_res']<=0.08].shape[0], 
      data_result[data_result['median_res']<=0.08].shape[0]/data_result.shape[0])
print(data_result[data_result['median_res']<=0.1].shape[0], 
      data_result[data_result['median_res']<=0.1].shape[0]/data_result.shape[0])

1984 0.20275932549821155
4494 0.459274399591211
6805 0.6954522227899846
7658 0.7826264690853347


In [39]:
data_result = data_result[['model_code', 'median_res']]
data_result.to_csv(curr_dir+"2020-11-02零售数据-B级车-误差中位数.csv", header=True, index=False)

### 计算A级车误差中位数

In [40]:
train_A = data_train[(data_train['level']=="缺失") | 
                     (data_train['level']=="A00") | 
                     (data_train['level']=="A0") | 
                     (data_train['level']=="A")]
print("筛选出A级以下的车训练集数量为:%d" % train_A.shape[0])
print("筛选出A级以下的车训练集--车型数量为:%d" % len(train_A['model_code'].unique()))
print("<<-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*>>")

test_A = data_test[(data_test['level']=="缺失") | 
                   (data_test['level']=="A00") | 
                   (data_test['level']=="A0") | 
                   (data_test['level']=="A")]
print("筛选出A级以下的车测试集数量为:%d" % test_A.shape[0])
print("筛选出A级以下的车测试集--车型数量为:%d" % len(test_A['model_code'].unique()))

筛选出A级以下的车训练集数量为:695928
筛选出A级以下的车训练集--车型数量为:18371
<<-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*>>
筛选出A级以下的车测试集数量为:175221
筛选出A级以下的车测试集--车型数量为:15517


In [41]:
# 从train_A中再抽取10%的数据作为validation set
val_A = train_A.sample(frac=0.1, random_state=0)
train_A = train_A.drop(val_A.index)
print(val_A.shape, train_A.shape)

print(train_A['level'].unique())
print(val_A['level'].unique())
print(test_A['level'].unique())


(69593, 311) (626335, 311)
['A' '缺失' 'A0' 'A00']
['A' 'A00' 'A0' '缺失']
['A0' 'A' '缺失' 'A00']


In [42]:
# 打乱数据
train_A = shuffle(train_A)
val_A = shuffle(val_A)
test_A = shuffle(test_A)

train_X_A = train_A.drop(['car_id', 'brand_code', 'brand_name', 'series_code', 
                          'series_name', 'model_code', 'model_name',  'color_name', 
                          'first_license_plate_date', 'province_name', 'city_name', 
                          'price', 'pay_time', 'residual', 'mileage_std', 'license_month', 
                          'new_car_price', 'count', 'series_level', 'level','level0','level1',
                          'level2','level3','level4','level5','level6','level7','level8'], axis=1)
train_y_A = train_A['residual']

val_X_A = val_A.drop(['car_id', 'brand_code', 'brand_name', 'series_code', 
                          'series_name', 'model_code', 'model_name',  'color_name', 
                          'first_license_plate_date', 'province_name', 'city_name', 
                          'price', 'pay_time', 'residual', 'mileage_std', 'license_month', 
                          'new_car_price', 'count', 'series_level', 'level','level0','level1',
                          'level2','level3','level4','level5','level6','level7','level8'], axis=1)
val_y_A = val_A['residual']

test_X_A = test_A.drop(['car_id', 'brand_code', 'brand_name', 'series_code', 
                          'series_name', 'model_code', 'model_name',  'color_name', 
                          'first_license_plate_date', 'province_name', 'city_name', 
                          'price', 'pay_time', 'residual', 'mileage_std', 'license_month', 
                          'new_car_price', 'count', 'series_level', 'level','level0','level1',
                          'level2','level3','level4','level5','level6','level7','level8'], axis=1)
test_y_A = test_A['residual']
print(train_X_A.shape, train_y_A.shape)
print(val_X_A.shape, val_y_A.shape)
print(test_X_A.shape, test_y_A.shape)

(626335, 282) (626335,)
(69593, 282) (69593,)
(175221, 282) (175221,)


In [43]:
import lightgbm as lgb

gbm = lgb.LGBMRegressor(boosting_type = 'gbdt', 
                        objective = 'regression',
                        learning_rate = 0.03, 
                        n_estimators = 3000, #可能会有所更改
                        max_depth = 9,
                        num_leaves = 200, 
                        subsample = 1, 
                        colsample_bytree = 0.3, 
                        min_child_samples = 50, 
                        n_jobs = 8,
                        verbose = 1)

gbm.fit(train_A[fearture_columns], 
        train_A[label],
        eval_set = (val_A[fearture_columns], val_A[label]),
        eval_metric = {'l1','l2'}, 
        early_stopping_rounds = 50, 
        verbose=True)


[1]	valid_0's l2: 0.0336999	valid_0's l1: 0.152018
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's l2: 0.0327049	valid_0's l1: 0.149665
[3]	valid_0's l2: 0.0313832	valid_0's l1: 0.146541
[4]	valid_0's l2: 0.0301534	valid_0's l1: 0.14357
[5]	valid_0's l2: 0.0289995	valid_0's l1: 0.140733
[6]	valid_0's l2: 0.0275835	valid_0's l1: 0.137176
[7]	valid_0's l2: 0.026903	valid_0's l1: 0.135405
[8]	valid_0's l2: 0.0256583	valid_0's l1: 0.132134
[9]	valid_0's l2: 0.0244237	valid_0's l1: 0.128837
[10]	valid_0's l2: 0.0232681	valid_0's l1: 0.125674
[11]	valid_0's l2: 0.0221569	valid_0's l1: 0.122557
[12]	valid_0's l2: 0.0211107	valid_0's l1: 0.119546
[13]	valid_0's l2: 0.0201939	valid_0's l1: 0.11684
[14]	valid_0's l2: 0.0194761	valid_0's l1: 0.114684
[15]	valid_0's l2: 0.0185964	valid_0's l1: 0.111983
[16]	valid_0's l2: 0.0181599	valid_0's l1: 0.110582
[17]	valid_0's l2: 0.0177429	valid_0's l1: 0.109253
[18]	valid_0's l2: 0.0173183	valid_0's l1: 0.107863
[19]	valid_0's

[154]	valid_0's l2: 0.00321593	valid_0's l1: 0.0434771
[155]	valid_0's l2: 0.00320972	valid_0's l1: 0.0434288
[156]	valid_0's l2: 0.00320352	valid_0's l1: 0.0433784
[157]	valid_0's l2: 0.00319791	valid_0's l1: 0.0433303
[158]	valid_0's l2: 0.00319472	valid_0's l1: 0.0433031
[159]	valid_0's l2: 0.00319107	valid_0's l1: 0.0432752
[160]	valid_0's l2: 0.00318495	valid_0's l1: 0.0432253
[161]	valid_0's l2: 0.00318227	valid_0's l1: 0.0432042
[162]	valid_0's l2: 0.00317655	valid_0's l1: 0.0431556
[163]	valid_0's l2: 0.00317092	valid_0's l1: 0.0431087
[164]	valid_0's l2: 0.0031657	valid_0's l1: 0.043065
[165]	valid_0's l2: 0.00316229	valid_0's l1: 0.0430365
[166]	valid_0's l2: 0.00315727	valid_0's l1: 0.0429953
[167]	valid_0's l2: 0.00315504	valid_0's l1: 0.0429768
[168]	valid_0's l2: 0.00315161	valid_0's l1: 0.0429486
[169]	valid_0's l2: 0.00314606	valid_0's l1: 0.0429034
[170]	valid_0's l2: 0.00314388	valid_0's l1: 0.0428844
[171]	valid_0's l2: 0.00314023	valid_0's l1: 0.0428541
[172]	valid_

[306]	valid_0's l2: 0.00285537	valid_0's l1: 0.0404908
[307]	valid_0's l2: 0.00285449	valid_0's l1: 0.0404831
[308]	valid_0's l2: 0.00285336	valid_0's l1: 0.0404735
[309]	valid_0's l2: 0.00285206	valid_0's l1: 0.0404629
[310]	valid_0's l2: 0.00285139	valid_0's l1: 0.040457
[311]	valid_0's l2: 0.00285	valid_0's l1: 0.0404454
[312]	valid_0's l2: 0.00284892	valid_0's l1: 0.0404359
[313]	valid_0's l2: 0.00284634	valid_0's l1: 0.0404147
[314]	valid_0's l2: 0.00284573	valid_0's l1: 0.040409
[315]	valid_0's l2: 0.00284461	valid_0's l1: 0.0403988
[316]	valid_0's l2: 0.00284222	valid_0's l1: 0.0403806
[317]	valid_0's l2: 0.00284078	valid_0's l1: 0.0403696
[318]	valid_0's l2: 0.00283959	valid_0's l1: 0.0403594
[319]	valid_0's l2: 0.0028384	valid_0's l1: 0.0403498
[320]	valid_0's l2: 0.00283762	valid_0's l1: 0.0403424
[321]	valid_0's l2: 0.00283706	valid_0's l1: 0.0403378
[322]	valid_0's l2: 0.00283643	valid_0's l1: 0.0403322
[323]	valid_0's l2: 0.00283557	valid_0's l1: 0.0403243
[324]	valid_0's 

[458]	valid_0's l2: 0.00272013	valid_0's l1: 0.0393444
[459]	valid_0's l2: 0.00271993	valid_0's l1: 0.0393429
[460]	valid_0's l2: 0.00271944	valid_0's l1: 0.0393388
[461]	valid_0's l2: 0.00271891	valid_0's l1: 0.0393343
[462]	valid_0's l2: 0.00271832	valid_0's l1: 0.0393288
[463]	valid_0's l2: 0.00271741	valid_0's l1: 0.0393216
[464]	valid_0's l2: 0.00271712	valid_0's l1: 0.0393196
[465]	valid_0's l2: 0.00271632	valid_0's l1: 0.039313
[466]	valid_0's l2: 0.00271532	valid_0's l1: 0.0393038
[467]	valid_0's l2: 0.00271482	valid_0's l1: 0.0392992
[468]	valid_0's l2: 0.00271439	valid_0's l1: 0.0392948
[469]	valid_0's l2: 0.00271399	valid_0's l1: 0.0392907
[470]	valid_0's l2: 0.00271364	valid_0's l1: 0.039287
[471]	valid_0's l2: 0.00271317	valid_0's l1: 0.0392831
[472]	valid_0's l2: 0.00271303	valid_0's l1: 0.0392818
[473]	valid_0's l2: 0.00271235	valid_0's l1: 0.039275
[474]	valid_0's l2: 0.00271219	valid_0's l1: 0.0392737
[475]	valid_0's l2: 0.00271191	valid_0's l1: 0.039271
[476]	valid_0'

[611]	valid_0's l2: 0.0026456	valid_0's l1: 0.0386901
[612]	valid_0's l2: 0.00264514	valid_0's l1: 0.038686
[613]	valid_0's l2: 0.0026447	valid_0's l1: 0.0386824
[614]	valid_0's l2: 0.00264401	valid_0's l1: 0.0386755
[615]	valid_0's l2: 0.00264393	valid_0's l1: 0.0386748
[616]	valid_0's l2: 0.00264333	valid_0's l1: 0.0386708
[617]	valid_0's l2: 0.0026428	valid_0's l1: 0.0386663
[618]	valid_0's l2: 0.00264218	valid_0's l1: 0.0386614
[619]	valid_0's l2: 0.00264163	valid_0's l1: 0.0386568
[620]	valid_0's l2: 0.00264138	valid_0's l1: 0.0386545
[621]	valid_0's l2: 0.002641	valid_0's l1: 0.0386507
[622]	valid_0's l2: 0.00264062	valid_0's l1: 0.0386478
[623]	valid_0's l2: 0.00263999	valid_0's l1: 0.0386417
[624]	valid_0's l2: 0.00263921	valid_0's l1: 0.0386357
[625]	valid_0's l2: 0.00263889	valid_0's l1: 0.038633
[626]	valid_0's l2: 0.00263823	valid_0's l1: 0.0386269
[627]	valid_0's l2: 0.00263808	valid_0's l1: 0.0386253
[628]	valid_0's l2: 0.00263773	valid_0's l1: 0.0386223
[629]	valid_0's l

...

[2405]	valid_0's l2: 0.00246893	valid_0's l1: 0.0370362
[2406]	valid_0's l2: 0.00246891	valid_0's l1: 0.0370362
[2407]	valid_0's l2: 0.00246892	valid_0's l1: 0.0370362
[2408]	valid_0's l2: 0.00246886	valid_0's l1: 0.0370358
[2409]	valid_0's l2: 0.00246884	valid_0's l1: 0.0370356
[2410]	valid_0's l2: 0.00246884	valid_0's l1: 0.0370354
[2411]	valid_0's l2: 0.00246884	valid_0's l1: 0.037035
[2412]	valid_0's l2: 0.00246885	valid_0's l1: 0.0370349
[2413]	valid_0's l2: 0.00246886	valid_0's l1: 0.0370349
[2414]	valid_0's l2: 0.00246882	valid_0's l1: 0.0370345
[2415]	valid_0's l2: 0.00246882	valid_0's l1: 0.0370344
[2416]	valid_0's l2: 0.00246882	valid_0's l1: 0.0370346
[2417]	valid_0's l2: 0.0024688	valid_0's l1: 0.0370345
[2418]	valid_0's l2: 0.00246877	valid_0's l1: 0.0370341
[2419]	valid_0's l2: 0.00246874	valid_0's l1: 0.0370339
[2420]	valid_0's l2: 0.00246873	valid_0's l1: 0.0370338
[2421]	valid_0's l2: 0.00246871	valid_0's l1: 0.0370337
[2422]	valid_0's l2: 0.00246867	valid_0's l1: 0.03

[2554]	valid_0's l2: 0.00246734	valid_0's l1: 0.0370177
[2555]	valid_0's l2: 0.00246733	valid_0's l1: 0.0370175
[2556]	valid_0's l2: 0.00246733	valid_0's l1: 0.0370176
[2557]	valid_0's l2: 0.00246735	valid_0's l1: 0.0370173
[2558]	valid_0's l2: 0.00246735	valid_0's l1: 0.0370174
[2559]	valid_0's l2: 0.00246739	valid_0's l1: 0.0370175
[2560]	valid_0's l2: 0.0024674	valid_0's l1: 0.0370174
[2561]	valid_0's l2: 0.00246732	valid_0's l1: 0.0370167
[2562]	valid_0's l2: 0.00246733	valid_0's l1: 0.0370168
[2563]	valid_0's l2: 0.00246729	valid_0's l1: 0.0370164
[2564]	valid_0's l2: 0.00246729	valid_0's l1: 0.0370165
[2565]	valid_0's l2: 0.00246725	valid_0's l1: 0.0370161
[2566]	valid_0's l2: 0.00246722	valid_0's l1: 0.0370156
[2567]	valid_0's l2: 0.0024672	valid_0's l1: 0.0370156
[2568]	valid_0's l2: 0.00246712	valid_0's l1: 0.0370146
[2569]	valid_0's l2: 0.0024671	valid_0's l1: 0.0370144
[2570]	valid_0's l2: 0.0024671	valid_0's l1: 0.0370145
[2571]	valid_0's l2: 0.00246711	valid_0's l1: 0.0370

[2704]	valid_0's l2: 0.00246562	valid_0's l1: 0.0369982
[2705]	valid_0's l2: 0.00246558	valid_0's l1: 0.0369981
[2706]	valid_0's l2: 0.00246561	valid_0's l1: 0.0369983
[2707]	valid_0's l2: 0.00246563	valid_0's l1: 0.0369985
[2708]	valid_0's l2: 0.00246562	valid_0's l1: 0.0369984
[2709]	valid_0's l2: 0.00246558	valid_0's l1: 0.0369981
[2710]	valid_0's l2: 0.00246556	valid_0's l1: 0.0369978
[2711]	valid_0's l2: 0.00246558	valid_0's l1: 0.0369981
[2712]	valid_0's l2: 0.00246556	valid_0's l1: 0.036998
[2713]	valid_0's l2: 0.00246556	valid_0's l1: 0.0369981
[2714]	valid_0's l2: 0.00246557	valid_0's l1: 0.0369985
[2715]	valid_0's l2: 0.00246555	valid_0's l1: 0.0369983
[2716]	valid_0's l2: 0.0024656	valid_0's l1: 0.0369985
[2717]	valid_0's l2: 0.00246552	valid_0's l1: 0.036998
[2718]	valid_0's l2: 0.00246549	valid_0's l1: 0.0369978
[2719]	valid_0's l2: 0.00246545	valid_0's l1: 0.0369976
[2720]	valid_0's l2: 0.00246544	valid_0's l1: 0.0369976
[2721]	valid_0's l2: 0.00246545	valid_0's l1: 0.036

[2855]	valid_0's l2: 0.00246443	valid_0's l1: 0.0369828
[2856]	valid_0's l2: 0.00246443	valid_0's l1: 0.0369824
[2857]	valid_0's l2: 0.00246441	valid_0's l1: 0.0369821
[2858]	valid_0's l2: 0.00246436	valid_0's l1: 0.0369818
[2859]	valid_0's l2: 0.00246434	valid_0's l1: 0.0369818
[2860]	valid_0's l2: 0.00246434	valid_0's l1: 0.0369818
[2861]	valid_0's l2: 0.00246433	valid_0's l1: 0.0369817
[2862]	valid_0's l2: 0.00246433	valid_0's l1: 0.0369816
[2863]	valid_0's l2: 0.00246431	valid_0's l1: 0.0369812
[2864]	valid_0's l2: 0.00246424	valid_0's l1: 0.0369809
[2865]	valid_0's l2: 0.00246423	valid_0's l1: 0.0369809
[2866]	valid_0's l2: 0.00246421	valid_0's l1: 0.0369806
[2867]	valid_0's l2: 0.00246419	valid_0's l1: 0.0369803
[2868]	valid_0's l2: 0.0024642	valid_0's l1: 0.0369804
[2869]	valid_0's l2: 0.0024642	valid_0's l1: 0.0369806
[2870]	valid_0's l2: 0.00246419	valid_0's l1: 0.0369805
[2871]	valid_0's l2: 0.00246422	valid_0's l1: 0.0369806
[2872]	valid_0's l2: 0.00246422	valid_0's l1: 0.03

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.3,
       learning_rate=0.03, max_depth=9, min_child_samples=50,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=3000,
       n_jobs=8, num_leaves=200, objective='regression', random_state=None,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1,
       subsample_for_bin=200000, subsample_freq=0, verbose=1)

In [44]:
ypred = gbm.predict(test_A[fearture_columns])
print_test_result(ypred, test_y_A)


预测误差在百分之 3以内数目为：91551,所占比例为：52.2489%
预测误差在百分之 5以内数目为：129822,所占比例为：74.0904%
预测误差在百分之 8以内数目为：158046,所占比例为：90.1981%
预测误差在百分之10以内数目为：166159,所占比例为：94.8282%


In [45]:
train_A['pred'] = gbm.predict(train_X_A)
train_A['err'] = abs(train_A['pred'] - train_A['residual'])
train_A[train_A['err']>=0.1].shape[0]

18341

In [47]:
train_A_new = train_A[train_A['err'] < 0.1]

train_X_A = train_A_new.drop(['car_id', 'brand_code', 'brand_name', 'series_code',
                              'series_name', 'model_code', 'model_name',  'color_name',
                              'first_license_plate_date', 'province_name', 'city_name',
                              'price', 'pay_time', 'residual', 'mileage_std', 'license_month',
                              'new_car_price', 'count', 'series_level', 'level', 'level0', 'level1',
                              'level2', 'level3', 'level4', 'level5', 'level6', 'level7', 'level8','pred', 'err'], axis=1)
train_y_A = train_A_new['residual']

In [48]:
gbm = lgb.LGBMRegressor(boosting_type = 'gbdt', 
                        objective = 'regression',
                        learning_rate = 0.03, 
                        n_estimators = 3000, #可能会有所更改
                        max_depth = 9,
                        num_leaves = 200, 
                        subsample = 1, 
                        colsample_bytree = 0.3, 
                        min_child_samples = 50, 
                        n_jobs = 8,
                        verbose = 1)

gbm.fit(train_X_A, 
        train_y_A,
        eval_set = (val_X_A, val_y_A),
        eval_metric = {'l1','l2'}, 
        early_stopping_rounds = 50, 
        verbose=True)

[1]	valid_0's l2: 0.0336939	valid_0's l1: 0.152
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's l2: 0.0326906	valid_0's l1: 0.14963
[3]	valid_0's l2: 0.0313649	valid_0's l1: 0.146498
[4]	valid_0's l2: 0.0301311	valid_0's l1: 0.143518
[5]	valid_0's l2: 0.0289748	valid_0's l1: 0.140673
[6]	valid_0's l2: 0.0275542	valid_0's l1: 0.137102
[7]	valid_0's l2: 0.0268718	valid_0's l1: 0.13533
[8]	valid_0's l2: 0.0256255	valid_0's l1: 0.13205
[9]	valid_0's l2: 0.0243899	valid_0's l1: 0.128747
[10]	valid_0's l2: 0.02323	valid_0's l1: 0.12557
[11]	valid_0's l2: 0.0221158	valid_0's l1: 0.122441
[12]	valid_0's l2: 0.0210677	valid_0's l1: 0.119425
[13]	valid_0's l2: 0.0201487	valid_0's l1: 0.116708
[14]	valid_0's l2: 0.0194303	valid_0's l1: 0.114547
[15]	valid_0's l2: 0.018549	valid_0's l1: 0.11184
[16]	valid_0's l2: 0.0181053	valid_0's l1: 0.11042
[17]	valid_0's l2: 0.0176937	valid_0's l1: 0.1091
[18]	valid_0's l2: 0.0172675	valid_0's l1: 0.107703
[19]	valid_0's l2: 0.0165

[153]	valid_0's l2: 0.00318962	valid_0's l1: 0.0432024
[154]	valid_0's l2: 0.00318735	valid_0's l1: 0.0431833
[155]	valid_0's l2: 0.00318098	valid_0's l1: 0.0431321
[156]	valid_0's l2: 0.00317512	valid_0's l1: 0.0430837
[157]	valid_0's l2: 0.00316961	valid_0's l1: 0.0430363
[158]	valid_0's l2: 0.00316674	valid_0's l1: 0.0430115
[159]	valid_0's l2: 0.00316345	valid_0's l1: 0.0429846
[160]	valid_0's l2: 0.00315732	valid_0's l1: 0.0429351
[161]	valid_0's l2: 0.00315496	valid_0's l1: 0.0429153
[162]	valid_0's l2: 0.00314932	valid_0's l1: 0.0428674
[163]	valid_0's l2: 0.00314426	valid_0's l1: 0.0428259
[164]	valid_0's l2: 0.00313919	valid_0's l1: 0.0427826
[165]	valid_0's l2: 0.00313563	valid_0's l1: 0.0427525
[166]	valid_0's l2: 0.00313047	valid_0's l1: 0.0427097
[167]	valid_0's l2: 0.00312821	valid_0's l1: 0.0426905
[168]	valid_0's l2: 0.00312473	valid_0's l1: 0.0426615
[169]	valid_0's l2: 0.00311951	valid_0's l1: 0.0426187
[170]	valid_0's l2: 0.00311733	valid_0's l1: 0.0425999
[171]	vali

[304]	valid_0's l2: 0.00284591	valid_0's l1: 0.0403116
[305]	valid_0's l2: 0.00284534	valid_0's l1: 0.0403068
[306]	valid_0's l2: 0.00284375	valid_0's l1: 0.0402932
[307]	valid_0's l2: 0.00284295	valid_0's l1: 0.040286
[308]	valid_0's l2: 0.00284197	valid_0's l1: 0.0402787
[309]	valid_0's l2: 0.00284086	valid_0's l1: 0.0402697
[310]	valid_0's l2: 0.00284025	valid_0's l1: 0.0402638
[311]	valid_0's l2: 0.00283918	valid_0's l1: 0.0402547
[312]	valid_0's l2: 0.00283817	valid_0's l1: 0.0402457
[313]	valid_0's l2: 0.00283631	valid_0's l1: 0.0402307
[314]	valid_0's l2: 0.0028358	valid_0's l1: 0.0402258
[315]	valid_0's l2: 0.00283432	valid_0's l1: 0.0402132
[316]	valid_0's l2: 0.00283166	valid_0's l1: 0.0401918
[317]	valid_0's l2: 0.00283063	valid_0's l1: 0.0401827
[318]	valid_0's l2: 0.00282941	valid_0's l1: 0.0401722
[319]	valid_0's l2: 0.00282851	valid_0's l1: 0.0401642
[320]	valid_0's l2: 0.00282764	valid_0's l1: 0.0401562
[321]	valid_0's l2: 0.00282709	valid_0's l1: 0.0401517
[322]	valid_

[454]	valid_0's l2: 0.00271653	valid_0's l1: 0.0391894
[455]	valid_0's l2: 0.00271638	valid_0's l1: 0.0391878
[456]	valid_0's l2: 0.00271546	valid_0's l1: 0.0391794
[457]	valid_0's l2: 0.00271482	valid_0's l1: 0.0391737
[458]	valid_0's l2: 0.00271446	valid_0's l1: 0.0391706
[459]	valid_0's l2: 0.00271429	valid_0's l1: 0.0391691
[460]	valid_0's l2: 0.00271404	valid_0's l1: 0.0391673
[461]	valid_0's l2: 0.00271377	valid_0's l1: 0.0391649
[462]	valid_0's l2: 0.00271334	valid_0's l1: 0.0391605
[463]	valid_0's l2: 0.00271217	valid_0's l1: 0.0391507
[464]	valid_0's l2: 0.00271193	valid_0's l1: 0.0391491
[465]	valid_0's l2: 0.00271126	valid_0's l1: 0.0391433
[466]	valid_0's l2: 0.00271023	valid_0's l1: 0.0391346
[467]	valid_0's l2: 0.00270974	valid_0's l1: 0.0391306
[468]	valid_0's l2: 0.00270925	valid_0's l1: 0.0391258
[469]	valid_0's l2: 0.00270886	valid_0's l1: 0.0391216
[470]	valid_0's l2: 0.00270844	valid_0's l1: 0.0391179
[471]	valid_0's l2: 0.00270788	valid_0's l1: 0.0391133
[472]	vali

[605]	valid_0's l2: 0.00264258	valid_0's l1: 0.0385257
[606]	valid_0's l2: 0.00264235	valid_0's l1: 0.0385234
[607]	valid_0's l2: 0.00264162	valid_0's l1: 0.0385183
[608]	valid_0's l2: 0.00264115	valid_0's l1: 0.0385141
[609]	valid_0's l2: 0.00264101	valid_0's l1: 0.0385128
[610]	valid_0's l2: 0.00264062	valid_0's l1: 0.038509
[611]	valid_0's l2: 0.00264023	valid_0's l1: 0.0385054
[612]	valid_0's l2: 0.0026397	valid_0's l1: 0.0385005
[613]	valid_0's l2: 0.00263937	valid_0's l1: 0.0384973
[614]	valid_0's l2: 0.00263887	valid_0's l1: 0.0384931
[615]	valid_0's l2: 0.00263878	valid_0's l1: 0.0384921
[616]	valid_0's l2: 0.00263845	valid_0's l1: 0.038489
[617]	valid_0's l2: 0.00263809	valid_0's l1: 0.0384854
[618]	valid_0's l2: 0.00263765	valid_0's l1: 0.0384807
[619]	valid_0's l2: 0.00263721	valid_0's l1: 0.0384771
[620]	valid_0's l2: 0.00263703	valid_0's l1: 0.038475
[621]	valid_0's l2: 0.0026367	valid_0's l1: 0.0384713
[622]	valid_0's l2: 0.00263632	valid_0's l1: 0.0384677
[623]	valid_0's

...

[2540]	valid_0's l2: 0.00247202	valid_0's l1: 0.0368499
[2541]	valid_0's l2: 0.002472	valid_0's l1: 0.0368498
[2542]	valid_0's l2: 0.00247196	valid_0's l1: 0.0368497
[2543]	valid_0's l2: 0.00247192	valid_0's l1: 0.0368492
[2544]	valid_0's l2: 0.00247192	valid_0's l1: 0.0368492
[2545]	valid_0's l2: 0.00247194	valid_0's l1: 0.0368493
[2546]	valid_0's l2: 0.00247194	valid_0's l1: 0.0368492
[2547]	valid_0's l2: 0.00247193	valid_0's l1: 0.0368493
[2548]	valid_0's l2: 0.00247194	valid_0's l1: 0.0368493
[2549]	valid_0's l2: 0.00247196	valid_0's l1: 0.0368495
[2550]	valid_0's l2: 0.00247197	valid_0's l1: 0.0368495
[2551]	valid_0's l2: 0.00247198	valid_0's l1: 0.0368496
[2552]	valid_0's l2: 0.00247193	valid_0's l1: 0.0368493
[2553]	valid_0's l2: 0.00247193	valid_0's l1: 0.0368495
[2554]	valid_0's l2: 0.0024719	valid_0's l1: 0.036849
[2555]	valid_0's l2: 0.00247191	valid_0's l1: 0.036849
[2556]	valid_0's l2: 0.00247191	valid_0's l1: 0.0368492
[2557]	valid_0's l2: 0.00247188	valid_0's l1: 0.03684

[2689]	valid_0's l2: 0.0024702	valid_0's l1: 0.0368318
[2690]	valid_0's l2: 0.0024702	valid_0's l1: 0.0368318
[2691]	valid_0's l2: 0.00247022	valid_0's l1: 0.036832
[2692]	valid_0's l2: 0.0024702	valid_0's l1: 0.0368319
[2693]	valid_0's l2: 0.00247018	valid_0's l1: 0.0368316
[2694]	valid_0's l2: 0.00247016	valid_0's l1: 0.0368314
[2695]	valid_0's l2: 0.00247017	valid_0's l1: 0.0368313
[2696]	valid_0's l2: 0.00247017	valid_0's l1: 0.0368313
[2697]	valid_0's l2: 0.00247017	valid_0's l1: 0.0368315
[2698]	valid_0's l2: 0.00247012	valid_0's l1: 0.0368311
[2699]	valid_0's l2: 0.00247011	valid_0's l1: 0.036831
[2700]	valid_0's l2: 0.0024701	valid_0's l1: 0.0368308
[2701]	valid_0's l2: 0.00247011	valid_0's l1: 0.0368311
[2702]	valid_0's l2: 0.0024701	valid_0's l1: 0.036831
[2703]	valid_0's l2: 0.00247012	valid_0's l1: 0.0368311
[2704]	valid_0's l2: 0.00247014	valid_0's l1: 0.0368312
[2705]	valid_0's l2: 0.00247012	valid_0's l1: 0.036831
[2706]	valid_0's l2: 0.00247015	valid_0's l1: 0.0368312
[

[2839]	valid_0's l2: 0.00246959	valid_0's l1: 0.0368222
[2840]	valid_0's l2: 0.00246958	valid_0's l1: 0.0368221
[2841]	valid_0's l2: 0.00246959	valid_0's l1: 0.0368221
[2842]	valid_0's l2: 0.0024696	valid_0's l1: 0.0368223
[2843]	valid_0's l2: 0.00246956	valid_0's l1: 0.036822
[2844]	valid_0's l2: 0.00246947	valid_0's l1: 0.0368214
[2845]	valid_0's l2: 0.0024695	valid_0's l1: 0.0368216
[2846]	valid_0's l2: 0.00246949	valid_0's l1: 0.0368214
[2847]	valid_0's l2: 0.00246947	valid_0's l1: 0.0368214
[2848]	valid_0's l2: 0.00246945	valid_0's l1: 0.0368211
[2849]	valid_0's l2: 0.00246947	valid_0's l1: 0.036821
[2850]	valid_0's l2: 0.00246946	valid_0's l1: 0.0368208
[2851]	valid_0's l2: 0.00246945	valid_0's l1: 0.0368207
[2852]	valid_0's l2: 0.00246947	valid_0's l1: 0.036821
[2853]	valid_0's l2: 0.00246946	valid_0's l1: 0.0368212
[2854]	valid_0's l2: 0.00246947	valid_0's l1: 0.0368211
[2855]	valid_0's l2: 0.00246948	valid_0's l1: 0.0368213
[2856]	valid_0's l2: 0.00246946	valid_0's l1: 0.03682

[2988]	valid_0's l2: 0.00246832	valid_0's l1: 0.0368076
[2989]	valid_0's l2: 0.00246831	valid_0's l1: 0.0368074
[2990]	valid_0's l2: 0.00246828	valid_0's l1: 0.0368071
[2991]	valid_0's l2: 0.0024683	valid_0's l1: 0.0368073
[2992]	valid_0's l2: 0.00246829	valid_0's l1: 0.0368071
[2993]	valid_0's l2: 0.00246821	valid_0's l1: 0.0368063
[2994]	valid_0's l2: 0.00246825	valid_0's l1: 0.0368066
[2995]	valid_0's l2: 0.00246818	valid_0's l1: 0.036806
[2996]	valid_0's l2: 0.00246814	valid_0's l1: 0.0368057
[2997]	valid_0's l2: 0.00246811	valid_0's l1: 0.0368055
[2998]	valid_0's l2: 0.0024681	valid_0's l1: 0.0368054
[2999]	valid_0's l2: 0.00246805	valid_0's l1: 0.0368049
[3000]	valid_0's l2: 0.00246794	valid_0's l1: 0.0368041
Did not meet early stopping. Best iteration is:
[3000]	valid_0's l2: 0.00246794	valid_0's l1: 0.0368041


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.3,
       learning_rate=0.03, max_depth=9, min_child_samples=50,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=3000,
       n_jobs=8, num_leaves=200, objective='regression', random_state=None,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1,
       subsample_for_bin=200000, subsample_freq=0, verbose=1)

In [50]:
ypred = gbm.predict(test_X_A)
print_test_result(ypred, test_y_A)

预测误差在百分之 3以内数目为：92217,所占比例为：52.6290%
预测误差在百分之 5以内数目为：130446,所占比例为：74.4466%
预测误差在百分之 8以内数目为：158207,所占比例为：90.2900%
预测误差在百分之10以内数目为：166143,所占比例为：94.8191%


In [51]:
by_model = test_A[['model_code', 'count', 'residual']]
by_model['pre_residual'] = ypred
by_model['pre_err'] = abs(by_model['residual'] - by_model['pre_residual'])
by_model['percent_err'] = by_model['pre_err']/by_model['residual']

data_result = pd.DataFrame(columns={'model_code':str, 'median_res':float})
data_result['model_code'] = list(by_model.groupby(['model_code'])['percent_err'].quantile(0.5).index)
data_result['median_res'] = list(by_model.groupby(['model_code'])['percent_err'].quantile(0.5))

print(data_result[data_result['median_res']<=0.03].shape[0], 
      data_result[data_result['median_res']<=0.03].shape[0]/data_result.shape[0])
print(data_result[data_result['median_res']<=0.05].shape[0], 
      data_result[data_result['median_res']<=0.05].shape[0]/data_result.shape[0])
print(data_result[data_result['median_res']<=0.08].shape[0], 
      data_result[data_result['median_res']<=0.08].shape[0]/data_result.shape[0])
print(data_result[data_result['median_res']<=0.1].shape[0], 
      data_result[data_result['median_res']<=0.1].shape[0]/data_result.shape[0])

2197 0.14158664690339628
5206 0.3355029967132822
9112 0.5872269124186377
10721 0.6909196365276793


In [52]:
data_result = data_result[['model_code', 'median_res']]
data_result.to_csv(curr_dir+"2020-11-02零售数据-A级车-误差中位数.csv", header = True, index=False)